In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [78]:
# Scrape website
# html=browser.html

# Find all the car listings on the page
# car_list=car_soup.find_all('div', class_='vehicle-card')

# Testing scraping for the first listing on the site
# car_model=car_soup.find('h2', class_='title')
# car_status=car_soup.find('p', class_='stock-type')
# car_price=car_soup.find('span', class_='primary-price')
# car_dealer=car_soup.find('div', class_='dealer-name')
# dealer_city=car_soup.find('div', class_='miles-from')

# print(car_model, car_price, car_dealer, car_status, dealer_city)


In [79]:
# Testing to make sure text that is stripped is correct
#model=car_model.get_text(strip=True)
#status=car_status.get_text(strip=True)
#price=car_price.get_text(strip=True)
#dealer=car_dealer.get_text(strip=True)
#city=dealer_city.get_text(strip=True)

#print(model, status, price, dealer, city)

In [2]:
# Set up Splinter
# Make sure chromedriver.exe is in PATH
browser= Browser('chrome')
# Store url and visit
base_url='https://www.cars.com/shopping/results/?makes[]=subaru&maximum_distance=50&models[]=subaru-outback&page=1&page_size=100&stock_type=all&zip=61606'
browser.visit(base_url)

# Create lists to store details
models = []
statuses = []
mileages=[]
prices = []
dealers = []
cities = []

# Function to reset lists
def clear_lists():
    global models, statuses, prices, dealers, cities, mileages
    models = []
    statuses = []
    prices = []
    dealers = []
    cities = []
    mileages = []

# Create a function to handle scraping current page
# Handle errors with ifs
def scrape_page():
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer = car.find('div', class_='dealer-name').find('strong')
        dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)

#WIP Trying to get next button to click
# Loop through all pages
while True:
    # Scrape the current page
    scrape_page()

    # Check if there is a "Next" button to go to the next page
    next_button = browser.find_by_css('a[aria-label="Next page"]')

    if next_button:
        try:
            next_button.click()  # Click on the next button
            print("Navigating to the next page...")
            time.sleep(30)  # Wait for the page to load, adjust as necessary
        except Exception as e:
            print(f"Exception occurred while navigating: {str(e)}")
            break  # Exit the loop if there's an issue navigating to the next page
    else:
        print("No more next button found. Exiting.")
        break  # Exit the loop if there's no next button

No more next button found. Exiting.


In [3]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities 
    }
car_df= pd.DataFrame(data)

In [4]:
car_df

,Model,Mileage (mi),Status,Price (USD),Dealer,City
0,2024 Subaru Outback Touring XT,N/A,New,"$45,033",Uftring Ford Subaru,"East Peoria, IL"
1,2013 Subaru Outback 2.5i Premium,"87,269 mi.",Used,"$12,950",Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)"
2,2022 Subaru Outback Onyx Edition XT,"46,577 mi.",Used,"$29,340",Uftring Chrysler Dodge Jeep RAM,"North Pekin, IL (6 mi.)"
3,2021 Subaru Outback Touring XT,"40,174 mi.",Used,"$29,000",Ford of Galesburg,"Galesburg, IL (45 mi.)"
4,2021 Subaru Outback Premium,"17,404 mi.",Used,"$27,998",CarMax Indianapolis - Offering Express Pickup ...,N/A
...,...,...,...,...,...,...
82,2023 Subaru Outback Onyx Edition,"11,888 mi.",Subaru Certified,"$30,509",Zeigler Subaru of Lafayette,"Lafayette, IN (148 mi.)"
83,2022 Subaru Outback Wilderness,"34,254 mi.",Used,"$29,997",Libertyville Hyundai,"Libertyville, IL (139 mi.)"
84,2016 Subaru Outback 2.5i Limited,"51,070 mi.",Used,"$18,995",INFINITI of Naperville,"Naperville, IL (105 mi.)"
85,2019 Subaru Outback 2.5i Premium,"28,160 mi.",Subaru Certified,"$22,806",Zeigler Subaru of Lafayette,"Lafayette, IN (148 mi.)"


In [5]:
# Checking to make sure df is formatted correctly
# print(car_df.to_string())

In [6]:
# Close the browser
browser.quit()

In [7]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                                  Model Mileage (mi)            Status  \
0        2024 Subaru Outback Touring XT          N/A               New   
1      2013 Subaru Outback 2.5i Premium   87,269 mi.              Used   
2   2022 Subaru Outback Onyx Edition XT   46,577 mi.              Used   
3        2021 Subaru Outback Touring XT   40,174 mi.              Used   
4           2021 Subaru Outback Premium   17,404 mi.              Used   
..                                  ...          ...               ...   
82     2023 Subaru Outback Onyx Edition   11,888 mi.  Subaru Certified   
83       2022 Subaru Outback Wilderness   34,254 mi.              Used   
84     2016 Subaru Outback 2.5i Limited   51,070 mi.              Used   
85     2019 Subaru Outback 2.5i Premium   28,160 mi.  Subaru Certified   
86     2016 Subaru Outback 2.5i Limited  113,796 mi.              Used   

   Price (USD)                                             Dealer  \
0      $45,033        

In [8]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
dtype: int64



In [9]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', 'Trim'
car_df[['Year', 'Manufacturer', 'Model', 'Trim']] = car_df['Model'].str.split(' ', 3, expand=True)

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean Mileage column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
    Year Manufacturer    Model             Trim  Mileage (mi)  \
0   2024       Subaru  Outback       Touring XT           NaN   
1   2013       Subaru  Outback     2.5i Premium       87269.0   
2   2022       Subaru  Outback  Onyx Edition XT       46577.0   
3   2021       Subaru  Outback       Touring XT       40174.0   
4   2021       Subaru  Outback          Premium       17404.0   
..   ...          ...      ...              ...           ...   
82  2023       Subaru  Outback     Onyx Edition       11888.0   
83  2022       Subaru  Outback       Wilderness       34254.0   
84  2016       Subaru  Outback     2.5i Limited       51070.0   
85  2019       Subaru  Outback     2.5i Premium       28160.0   
86  2016       Subaru  Outback     2.5i Limited      113796.0   

              Status  Price (USD)  \
0                New      45033.0   
1               Used      12950.0   
2               Used      29340.0   
3               Used      29000.0   
4               U

C:\Users\awsom\AppData\Local\Temp\ipykernel_520\3127507735.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [10]:
car_df

,Year,Manufacturer,Model,Trim,Mileage (mi),Status,Price (USD),Dealer,City
0,2024,Subaru,Outback,Touring XT,NaN,New,45033.0,Uftring Ford Subaru,"East Peoria, IL"
1,2013,Subaru,Outback,2.5i Premium,87269.0,Used,12950.0,Sam Leman Toyota - Bloomington,"Bloomington, IL (38 mi.)"
2,2022,Subaru,Outback,Onyx Edition XT,46577.0,Used,29340.0,Uftring Chrysler Dodge Jeep RAM,"North Pekin, IL (6 mi.)"
3,2021,Subaru,Outback,Touring XT,40174.0,Used,29000.0,Ford of Galesburg,"Galesburg, IL (45 mi.)"
4,2021,Subaru,Outback,Premium,17404.0,Used,27998.0,CarMax Indianapolis - Offering Express Pickup ...,N/A
...,...,...,...,...,...,...,...,...,...
82,2023,Subaru,Outback,Onyx Edition,11888.0,Subaru Certified,30509.0,Zeigler Subaru of Lafayette,"Lafayette, IN (148 mi.)"
83,2022,Subaru,Outback,Wilderness,34254.0,Used,29997.0,Libertyville Hyundai,"Libertyville, IL (139 mi.)"
84,2016,Subaru,Outback,2.5i Limited,51070.0,Used,18995.0,INFINITI of Naperville,"Naperville, IL (105 mi.)"
85,2019,Subaru,Outback,2.5i Premium,28160.0,Subaru Certified,22806.0,Zeigler Subaru of Lafayette,"Lafayette, IN (148 mi.)"


In [12]:
print(car_df.to_string())

    Year Manufacturer    Model             Trim  Mileage (mi)            Status  Price (USD)                                                           Dealer                        City
0   2024       Subaru  Outback       Touring XT           NaN               New      45033.0                                              Uftring Ford Subaru             East Peoria, IL
1   2013       Subaru  Outback     2.5i Premium       87269.0              Used      12950.0                                   Sam Leman Toyota - Bloomington    Bloomington, IL (38 mi.)
2   2022       Subaru  Outback  Onyx Edition XT       46577.0              Used      29340.0                                  Uftring Chrysler Dodge Jeep RAM     North Pekin, IL (6 mi.)
3   2021       Subaru  Outback       Touring XT       40174.0              Used      29000.0                                                Ford of Galesburg      Galesburg, IL (45 mi.)
4   2021       Subaru  Outback          Premium       17404.0         

In [11]:
car_df.to_excel("Outback.xlsx")